# Compare pandas with polars and ducks
* artificial dataframe of genetic data? Is there other data with general interest
* what ARE the time expensive operations?
* I will use the `ko` parquet file
* I want to reproduce queries from TPC-H benchmark
  * fireducks developed codes in https://github.com/fireducks-dev/polars-tpch/tree/fireducks_20250204
  

In [1]:
import pandas as pd
import polars as pl


import utils_pandas as upd
import utils_polars as upl
import utils_fireducks as ufd

%load_ext autoreload
%autoreload 2


In [2]:

def q1_pandas() -> None:
    # first call one time to cache in case we don't include the IO times
    lineitem = upd.get_line_item_ds()

    def query():
        lineitem = upd.get_line_item_ds()

        q_final = (
            lineitem[lineitem["abundance"] >= 1000]
            .groupby("ref_code")
            .agg(
                sum_abundance=("abundance", "sum"),
                avg_abundance=("abundance", "mean"),
                count_abundance=("abundance", "count"),
            )
            .sort_values("sum_abundance")
        )
        # q_final = (
        #     lineitem[lineitem["l_shipdate"] <= date(1998, 9, 2)]
        #     .assign(
        #         disc_price=lambda df: df["l_extendedprice"] * (1 - df["l_discount"])
        #     )
        #     .assign(charge=lambda df: df["disc_price"] * (1 + df["l_tax"]))
        #     .groupby(["l_returnflag", "l_linestatus"], as_index=False)
        #     .agg(
        #         sum_qty=("l_quantity", "sum"),
        #         sum_base_price=("l_extendedprice", "sum"),
        #         sum_disc_price=("disc_price", "sum"),
        #         sum_charge=("charge", "sum"),
        #         avg_qty=("l_quantity", "mean"),
        #         avg_price=("l_extendedprice", "mean"),
        #         avg_disc=("l_discount", "mean"),
        #         count_order=("l_returnflag", "count"),
        #     )
        #     .sort_values(["l_returnflag", "l_linestatus"])
        # )

        return q_final
    
    upd.run_query(Q_NUM, query)


def q1_polars() -> None:
    lineitem = upl.get_line_item_ds()

    var1 = 1000
    q_final = (
        lineitem.filter(pl.col("abundance") >= var1)
        .group_by("ref_code")
        .agg(
            pl.sum("abundance").alias("sum_abundance"),
            pl.mean("abundance").alias("avg_abundance"),
            pl.len().alias("count"),
        )
        .sort("sum_abundance")
    )
    # q_final = (
    #     lineitem.filter(pl.col("l_shipdate") <= var1)
    #     .group_by("l_returnflag", "l_linestatus")
    #     .agg(
    #         pl.sum("l_quantity").alias("sum_qty"),
    #         pl.sum("l_extendedprice").alias("sum_base_price"),
    #         (pl.col("l_extendedprice") * (1.0 - pl.col("l_discount")))
    #         .sum()
    #         .alias("sum_disc_price"),
    #         (
    #             pl.col("l_extendedprice")
    #             * (1.0 - pl.col("l_discount"))
    #             * (1.0 + pl.col("l_tax"))
    #         )
    #         .sum()
    #         .alias("sum_charge"),
    #         pl.mean("l_quantity").alias("avg_qty"),
    #         pl.mean("l_extendedprice").alias("avg_price"),
    #         pl.mean("l_discount").alias("avg_disc"),
    #         pl.len().alias("count_order"),
    #     )
    #     .sort("l_returnflag", "l_linestatus")
    # )

    upl.run_query(Q_NUM, q_final)


def q1_fireducks():
    # first call one time to cache in case we don't include the IO times
    ufd.get_line_item_ds()

    def query():
        lineitem = upd.get_line_item_ds()

        q_final = (
            lineitem[lineitem["abundance"] >= 1000]
            .groupby("ref_code")
            .agg(
                sum_abundance=("abundance", "sum"),
                avg_abundance=("abundance", "mean"),
                count_abundance=("abundance", "count"),
            )
            .sort_values("sum_abundance")
        )

        return q_final

    ufd.run_query(Q_NUM, query)


In [3]:
# df = upd.get_line_item_ds()
# df.info(), df.nunique()

In [4]:
Q_NUM = 1

In [5]:
q1_pandas()
q1_polars()


Code block 'Run pandas query 1' took: 0.08028 s
Code block 'Run polars query 1' took: 0.00321 s


In [6]:
%load_ext fireducks.pandas
import pandas as pd

q1_fireducks()


Code block 'Run fireducks query 1' took: 0.00805 s
